# Data Cleaning

Import libraries/packages + raw data

In [ ]:
import pandas as pd
import numpy as np

# Import Data
df = pd.read_excel("../data/raw/ORN_Data.xlsx")

Import, subset, and rename columns --> only include patients with Radiation Therapy

In [ ]:
# Subset to only include relevant rows
df = df[:329]
## Rename for interpretability
df = df.rename(columns={"ORN": "ORN_type", "ORN ": "ORN"})
##Subset relevant columns
relevant_cols = [
    # Patient Characteristics
    "AGE",
    "GENDER (0=female, 1=male)",
    "BMI",
    "ASA, 0= ASA I or II, 1=ASA III",
    "DM(0=no, 1=yes)",
    "PRERT( pre-op RT)",
    "PRECT(pre-op chemotherapy)",
    "PRIOREX( prior opearion at same side, 0=no, 1=yes)",
    # Tumor Characteristics
    "SECONDPRIMARY(0=no, 1=yes)",
    "RECUR(0=primary, 1=recurrence)",
    "SITE(1= mouth floor,  2=buccal, 3=retromolar, 4=gum, 5=tongue , 6=lip)",
    "T(0=T1or2, 1=T3or4)",
    "N(0= (-), 1=(+))",
    "OVERALLSTAGE(1= stage1, 2=stage2/3, 3=stage4)",
    "DEFECTTYPE(0= intraoral only, 1=composite defect)",
    "JEWER(Jewer's classification, 0=C, 1=L, 3=LC, 5=LCL)",
    # Peri-op
    "OPTIME",
    "LENGTH(defect length)",
    "OSTEOTOMY(no. of osteostomy of the fibula bone)",
    "PLATE(0=mini plate 1=reconstruction plate 2=preformed plate)",
    "FLAP(0=OSC flap, 1=chimeric flap with muscle)",
    "BT(intra-op blood transfusion, 0=no, 1=yes)",
    "ISCHEMICTIME",
    ## Post-op
    "ADMISSION( hospitalization days)",
    "HGB(post-op hemoglobin)",
    "ALB(post-op albumin)",
    "REOPEN",
    "WOUNDINF(post-op wound infection)",
    "EXPOSURE( plate exposure)",
    "EXPOSUREFU( time from op to plate exposure)",
    "MEDEXPOSURE( treat plate exposure with medication only)",
    "TXEXPOSURE(treatments other than medication, 1=plate removal, 2=another flap)",
    "POSTRT(post-op RT)",
    "POSTCT(post-op chemotherapy)",
    "ORN",
]
df_sub = df[relevant_cols]

##Rename columns for readability
df_renamed = df_sub.rename(
    columns={
        "REOPEN": "REOPEN",
        "ADMISSION( hospitalization days)": "ADMISSION",
        "MEDEXPOSURE( treat plate exposure with medication only)": "MEDEXPOSURE",
        "TXEXPOSURE(treatments other than medication, 1=plate removal, 2=another flap)": "TXEXPOSURE",
        "PRIOREX( prior opearion at same side, 0=no, 1=yes)": "PRIOREX",
        "EXPOSUREFU( time from op to plate exposure)": "EXPOSUREFU",
        "GENDER (0=female, 1=male)": "GENDER",
        "ASA, 0= ASA I or II, 1=ASA III": "ASA",
        "DM(0=no, 1=yes)": "DM",
        "RECUR(0=primary, 1=recurrence)": "RECUR",
        "SECONDPRIMARY(0=no, 1=yes)": "SP",
        "SITE(1= mouth floor,  2=buccal, 3=retromolar, 4=gum, 5=tongue , 6=lip)": "SITE",
        "T(0=T1or2, 1=T3or4)": "T",
        "N(0= (-), 1=(+))": "N",
        "OVERALLSTAGE(1= stage1, 2=stage2/3, 3=stage4)": "STAGE",
        "DEFECTTYPE(0= intraoral only, 1=composite defect)": "DEFECT_TYPE",
        "JEWER(Jewer's classification, 0=C, 1=L, 3=LC, 5=LCL)": "JEWER",
        "LENGTH(defect length)": "LENGTH",
        "OSTEOTOMY(no. of osteostomy of the fibula bone)": "OSTEOTOMY",
        "PLATE(0=mini plate 1=reconstruction plate 2=preformed plate)": "PLATE",
        "FLAP(0=OSC flap, 1=chimeric flap with muscle)": "FLAP",
        "HGB(post-op hemoglobin)": "HGB",
        "ALB(post-op albumin)": "ALB",
        "BT(intra-op blood transfusion, 0=no, 1=yes)": "BT",
        "PRERT( pre-op RT)": "PRERT",
        "POSTRT(post-op RT)": "POSTRT",
        "PRECT(pre-op chemotherapy)": "PRECT",
        "POSTCT(post-op chemotherapy)": "POSTCT",
        "WOUNDINF(post-op wound infection)": "WOUNDINF",
        "EXPOSURE( plate exposure)": "EXPOSURE",
    }
)

# Subset only RT patients
df_renamed = df_renamed[(df_renamed["POSTRT"] == 1) | (df_renamed["PRERT"] == 1)]
##Replace 999 with NA
df_renamed = df_renamed.replace(999, np.nan)
## Final shape
print(f"Cleaned Shape: {df_renamed.shape}")

Cleaned Shape: (275, 35)


Simplify features

In [ ]:
# Change the single '2' entry for exposure to a '1' for simplicity
df_renamed["EXPOSURE"] = df_renamed["EXPOSURE"].replace({2: 1})
# Change two '3' entries in OSTEOTOMY to '2' for simplicity
df_renamed["OSTEOTOMY"] = df_renamed["OSTEOTOMY"].replace({3.0: 2.0})
# Rename ASA entries to make binary nature more clear (2-->0, 3-->1)
df_renamed["ASA"] = df_renamed["ASA"].replace({2: 0, 3: 1})
# Combine 2 'JEWER 0' (C) with 71 'JEWER 3' (LC) b/c of low frequency
df_renamed["JEWER"] = df_renamed["JEWER"].replace({0.0: 3.0})

Export

In [ ]:
df_renamed.to_parquet("../data/raw/orn_clean.parquet")